# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
!pip install -U sentence-transformers

  Using cached https://files.pythonhosted.org/packages/8f/e9/c2b4c823b3959d475a570c1bd2df4125478e2e37b96fb967a87933ae7134/transformers-4.18.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d8/87/b37ebc960d0a85e10785a1a92d796edbd975840bee150a9ae3ba5d7a0250/sentencepiece-0.1.99.tar.gz
  Using cached https://files.pythonhosted.org/packages/a4/54/81b1c3c574a1ffde54b0c82ed2a37d81395709cdd5f50e59970aeed5d95e/torch-1.10.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c8/df/1b454741459f6ce75f86534bdad42ca17291b14a83066695f7d2c676e16c/huggingface_hub-0.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/57/da0cb8e40437f88630769164a66afec8af294ff686661497b6c88bc08556/tokenizers-0.12.1.tar.gz
  Installing build dependencies ... done
  Using cached https://files.pythonhosted.org/packages/84/ce/8916d10ef537f3f3b046843255f9799504aa41862bfa87844b9bdc5361cd/filelock-3.4.1-py3-none-any.whl
  Using cached https

In [2]:
# import libraries
import re
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from gensim.models import Word2Vec

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table(con=engine,table_name="disaster_tweet")
X = df['message']
Y = df.iloc[:,3:].values

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    clean_tokens = [w for w in clean_tokens if not w.lower() in stop_words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf_transformer', TfidfTransformer()),
    ('classifier',MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.75, random_state=42)

pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
output_category = list(df.iloc[:,3:].columns)
for i in range(Y_test.shape[1]):
    print(output_category[i])
    print(classification_report(Y_test.T[i], Y_pred.T[i]))

related
             precision    recall  f1-score   support

          0       0.60      0.34      0.44      4608
          1       0.82      0.93      0.87     15053

avg / total       0.77      0.79      0.77     19661

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93     16288
          1       0.77      0.40      0.53      3373

avg / total       0.87      0.88      0.86     19661

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19569
          1       0.00      0.00      0.00        92

avg / total       0.99      1.00      0.99     19661

aid_related
             precision    recall  f1-score   support

          0       0.73      0.86      0.79     11514
          1       0.73      0.55      0.63      8147

avg / total       0.73      0.73      0.72     19661

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.98      1.00      0.99     19209
          1       1.00      0.01      0.02       452

avg / total       0.98      0.98      0.97     19661

missing_people
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     19438
          1       1.00      0.01      0.03       223

avg / total       0.99      0.99      0.98     19661

refugees
             precision    recall  f1-score   support

          0       0.97      1.00      0.98     19014
          1       0.60      0.03      0.05       647

avg / total       0.96      0.97      0.95     19661

death
             precision    recall  f1-score   support

          0       0.96      1.00      0.98     18779
          1       0.72      0.14      0.24       882

avg / total       0.95      0.96      0.95     19661

other_aid
             precision    recall  f1-score   support

          0       0.87      1.00      0.93    

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {
    'count_vectorizer__ngram_range': ((1, 1), (1, 2)),
    'classifier__estimator__n_estimators': [2,5,10,20],
    'classifier__estimator__min_samples_split': [2,5,10,20,40]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train,Y_train)

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf_transformer', TfidfTransformer()),
    ('classifier',MultiOutputClassifier(RandomForestClassifier())),
])

In [294]:
vectorizer = CountVectorizer(tokenizer=tokenize)
vectorizer.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7f0d611b5bf8>, vocabulary=None)

In [8]:
# class Vectorizer(BaseEstimator, TransformerMixin):
#     def fit(self, x, y=None, min_count, vector_size):
#         self.min_count=min_count
#         self.vector_size=vector_size
#         return self

#     def transform(self, X):
#         w2v = word2vec(sentences, min_count=self.min_count, size = self.vector_size)
#         return pd.DataFrame(X_tagged)

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                    return True
            except:
                return False
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

class StartingNounExtractor(BaseEstimator, TransformerMixin):
    def starting_noun(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['NN', 'NNS', 'NNP', 'NNPS']:
                    return True
            except:
                return False
        return False
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_noun)
        return pd.DataFrame(X_tagged)

pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor()),
        ('starting_noun', StartingNounExtractor()),
    ])),

    ('classifier', MultiOutputClassifier(RandomForestClassifier())),
])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.75, random_state=42)

pipeline.fit(X_train,Y_train)
Y_pred = pipeline.predict(X_test)

In [9]:
output_category = list(df.iloc[:,3:].columns)
for i in range(Y_test.shape[1]):
    print(output_category[i])
    print(classification_report(Y_test.T[i], Y_pred.T[i]))

related
             precision    recall  f1-score   support

          0       0.60      0.33      0.42      4608
          1       0.82      0.93      0.87     15053

avg / total       0.77      0.79      0.77     19661

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93     16288
          1       0.78      0.35      0.48      3373

avg / total       0.86      0.87      0.85     19661

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19569
          1       0.00      0.00      0.00        92

avg / total       0.99      1.00      0.99     19661

aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79     11514
          1       0.76      0.52      0.61      8147

avg / total       0.73      0.73      0.72     19661

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.